<a href="https://colab.research.google.com/github/dyou-wbg/meta_processing/blob/main/50k_sample__char_based_distance_knn_deduping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import pandas as pd
from google.colab import drive
import csv
import os
import re
import pandas as pd
import numpy as np

directory = '/content/drive/MyDrive/finalMetaFile'

Mounted at /content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)


<ipython-input-3-1390d13c0cdd>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
# can skip if directly grab the 50k sample file
df = pd.read_csv('/content/drive/MyDrive/All_titles.csv')
df.rename(columns={'0': 'title'}, inplace=True)
df=df.drop(columns=['Unnamed: 0'])
df=df.sample(n=50000, random_state=321).dropna()

# deselect non-english titles

In [ ]:

import nltk
nltk.download('punkt')
from nltk import word_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# can skip if directly grab the 50k sample file
# select when title is at least >=2 character and have 70% of them in english
def remove_non_english(df):
  def is_english(text):
    words = word_tokenize(text)
    english_word_count = sum(len(word) >= 2 for word in words)
    return english_word_count / len(words) >= 0.7

  df_filtered = df[df['title'].apply(is_english)]
  return df_filtered

df_english_titles = remove_non_english(df.copy())

print("Original DataFrame:")
print(len(df))
print("\nDataFrame with English titles only:")
print(len(df_english_titles))

Original DataFrame:
49898

DataFrame with English titles only:
48173


In [ ]:
# can skip if directly grab the 50k sample file
df_english_titles = df_english_titles[~df_english_titles['title'].str.contains('و|д|я', regex=True)]

In [ ]:
len(df_english_titles)

47554

In [ ]:
df_english_titles = pd.read_csv('/content/drive/MyDrive/finalMetaFile/50k_sample.csv')

In [ ]:
df_english_titles=df_english_titles[~df_english_titles['title'].str.lower().isin(["conclusion", "poetry","introduction","front matter","editorial","back matter","correspondence","ist of Abbreviations","Book Reviews","Notifiable Diseases and Mortality Tables"])]
len(df_english_titles)

47526

In [ ]:
df_english_titles.to_csv(os.path.join(directory, '50k_sample.csv'), index=False)

In [ ]:
len(df_english_titles)/50000

0.95052

# from here

In [ ]:
df_english_titles = pd.read_csv('/content/drive/MyDrive/finalMetaFile/50k_sample.csv')

# (base model) char freq based vector and KDtree

In [ ]:
from string import printable
from collections import Counter

printable_dict = {c:i for i,c in enumerate(printable)}

def vectorize(title):
    cnt = Counter(title)
    array = [0 for _ in printable]+[0] # Last position is for nonprintable characters
    for c,n in cnt.items():
        if c in printable_dict:
            array[printable_dict[c]]=n
        else:
            array[-1]+=n # Update last position if it's not printable
    return np.array(array)

vects = []
for title in df_english_titles.title:
    vects.append(vectorize(title))

In [ ]:
from sklearn.neighbors import KDTree
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_english_titles['title'])



In [ ]:
printable_dict = {c:i for i,c in enumerate(printable)}


def vectorize(title):
    cnt = Counter(title)
    array = [0 for _ in printable]+[0] # Last position is for nonprintable characters
    for c,n in cnt.items():
        if c in printable_dict:
            array[printable_dict[c]]=n
        else:
            array[-1]+=n # Update last position if it's not printable
    return np.array(array)

vects = []
for title in df_english_titles.title:
    vects.append(vectorize(title))

In [ ]:

from string import printable
from collections import Counter



# kdtree
from sklearn.neighbors import KDTree
tree = KDTree(np.array(vects), leaf_size=10, metric='cityblock')


for i in range(len(vects)):
    dist, ind = tree.query(vects[i:i+1], k=2)
    #print(df_english_titles.iloc[i].title)
    print(df_english_titles.iloc[ind[0]].title)

Streaming output truncated to the last 5000 lines.
4061     Study on the Real-time Control in Transit    
Name: title, dtype: object
45860    Propagation of aggregate demand, firm specific and policy shocks in an economy with labour flows and between firms of different sizes
29223    Effects of stored feed cropping systems and farm size on the profitability of Maine organic dairy farm simulations.                  
Name: title, dtype: object
45861    WIDER Working Paper 2016/139 Trade, poverty, and social protection in developing countries
16502    Factor proportions, relative efficiency and developing countries' trade                   
Name: title, dtype: object
45862    The Use of the Value Chain Concept to Public Policies Design and Evaluation: A Methodological Approach for the Brazilian Software Sector     
21631    Recovering from the crisis through social dialogue in the new EU Member States: the case of Bulgaria, the Czech Republic, Poland and Slovenia
Name: title, dtype: objec

In [ ]:
threshold = 10
tree = KDTree(np.array(vects), leaf_size=10, metric='cityblock')

def find_similar_titles(df, tree, threshold=None):
  all_similar_titles = []

  for i in range(len(vects)):
    dist, ind = tree.query(vects[i:i+1], k=2)
    if threshold is not None:
      similar_titles = [df_english_titles.iloc[j].title for j in ind[0] if j != i and dist[0][0] <= threshold]
    else:
      similar_titles = [df_english_titles.iloc[j].title for j in ind[0] if j != i]
    all_similar_titles.append([df_english_titles.iloc[i].title] + similar_titles)

  return all_similar_titles

similar_titles_data = find_similar_titles(df_english_titles, tree, threshold=threshold)

with open("similar_titles_kdtree.csv", "w", newline='') as f:
  writer = csv.writer(f)
  writer.writerow(["Original Title", "Similar Title 1", "Similar Title 2"])
  for title, similar_list in similar_titles_data:
    writer.writerow([title] + similar_list[1:])  # Exclude the first title (itself)

print("Similar titles saved to similar_titles_kdtree.csv")

In [ ]:

%%time
all_similar_titles_base = []

tree = KDTree(np.array(vects), leaf_size=8, metric='cityblock')

for i in range(len(vects)):
    current_title = df_english_titles.iloc[i].title
    dist, ind = tree.query(vects[i:i+1], k=3)
    nearest_titles = [df_english_titles.iloc[j].title for j in ind[0]]

    similar_titles = []
    for neighbor_title in nearest_titles:
        if neighbor_title != current_title:
            similarity_score = cosine_similarity(tfidf_matrix[i], tfidf_matrix[ind[0][nearest_titles.index(neighbor_title)]])
            if similarity_score > 0.9:
                similar_titles.append(neighbor_title)

    if similar_titles:  # If any similar titles found, store in all_similar_titles
        all_similar_titles_base.append({"title": current_title, "similar_titles": similar_titles})


print(f"Found {len(similar_titles)} similar titles out of {len(df_english_titles)}")

all_similar_titles_base=pd.DataFrame(all_similar_titles_base)
all_similar_titles_base.to_csv(os.path.join(directory, '50k_sample_similar_titles_byKDtree_consineSim.csv'), index=False)

# base 2

In [ ]:
%%time
from collections import defaultdict
threshold = 10
min_length = 20

def find_similar_titles(df, vectors, threshold, min_length):
  all_similar_titles = defaultdict(list)
  for i, v1 in enumerate(vectors):
    if sum(v1[:-1]) > min_length:
      for j, v2 in enumerate(vectors):
        if sum(v2[:-1]) > min_length and i != j:
          l1 = np.linalg.norm(v1[:-1] - v2[:-1], ord=1)
          if l1 < threshold:
            t1 = df.iloc[i].title
            t2 = df.iloc[j].title
            all_similar_titles[t1].append(t2)

  return all_similar_titles


similar_titles = find_similar_titles(df_english_titles, vects, threshold, min_length)


for title, similar_list in similar_titles.items():
  print(title)
  for similar_title in similar_list:
    print(f"\t- {similar_title}")
  print('-' * 50)

similar_titles=pd.DataFrame(similar_titles)
similar_titles.to_csv(os.path.join(directory, '50k_sample_similar_titles_byKDtree_l1_10.csv'), index=False)

# benchmark 1: simple tf-idf and consine similarity

In [ ]:
%%time
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_english_titles['title'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

num_titles = len(df_english_titles)
for i in range(num_titles):
    current_title = df_english_titles.iloc[i]['title']
    similarities = cosine_sim[i]  # Similarity scores for current title with all others

    # Perform comparison and actions based on similarities:
    for j in range(num_titles):
        if i != j:  # Skip comparison with itself
            other_title = df_english_titles.iloc[j]['title']
            similarity_score = similarities[j]

            # Customize actions here:
            if similarity_score > 0.85 & similarity_score < 0.999 :  # Example threshold for high similarity
                print(f"{current_title} & {other_title} (score: {similarity_score:.4f})")
            # Perform other comparisons or actions as needed

            if similar_titles:  # If any similar titles found
              all_similar_titles.append({"title": current_title,"similar_titles": similar_titles})

CENTRAL EUROPEAN REVIEW OF ECONOMICS AND MANAGEMENT & CENTRAL EUROPEAN REVIEW OF ECONOMICS AND MANAGEMENT (score: 1.0000)
Defense Briefs & Defense Briefs (score: 1.0000)
Relative Wage Variability: Monetary Policy and the Labor Market & Relative Wage Variability: Monetary Policy and the Labor Market (score: 1.0000)
Editor's introduction & Editor's Introduction (score: 1.0000)
Editor's introduction & Editor's Introduction (score: 1.0000)
Calcutta Diary & Calcutta Diary (score: 1.0000)
Calcutta Diary & Calcutta Diary (score: 1.0000)
Calcutta Diary & Calcutta Diary (score: 1.0000)
JURISPRUDENCE FINANCIÈRE: RÉPERTOIRE ANALYTIQUE & JURISPRUDENCE FINANCIÈRE: Répertoire analytique (score: 1.0000)
Could the Corporations Become Engines for International Cooperation? & COULD THE CORPORATIONS BECOME ENGINES FOR INTERNATIONAL COOPERATION (score: 1.0000)
Identifying the Monetary Transmission Mechanism Using Structural Breaks & Identifying the monetary transmission mechanism using structural breaks (

In [ ]:
all_similar_titles_tfidf = []

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df_english_titles['title'])
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

num_titles = len(df_english_titles)
for i in range(num_titles):
    current_title = df_english_titles.iloc[i]['title']
    similarities = cosine_sim[i]
    similar_titles = []

    for j in range(num_titles):
        if i != j:  # Skip comparison with itself
            other_title = df_english_titles.iloc[j]['title']
            similarity_score = similarities[j]

            # Actions based on similarity score:
            if similarity_score > 0.85:
                print(f"{current_title} is highly similar to {other_title} (score: {similarity_score:.4f})")
                similar_titles.append(other_title)  # Add to similar titles list

    if similar_titles:  # If any similar titles found, store in all_similar_titles
        all_similar_titles_tfidf.append({"title": current_title, "similar_titles": similar_titles})

all_similar_titles_tfidf=pd.DataFrame(all_similar_titles_tfidf)
all_similar_titles_tfidf.to_csv(os.path.join(directory, '50k_sample_similar_titles_bytfidf_consineSim.csv'), index=False)

# benchmark 2 : BERT

In [ ]:
from sklearn.neighbors import KDTree
from transformers import BertModel, BertTokenizer
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-mpnet-base-v2')


def get_encoded_vectors(titles):
  encoded_inputs = model.encode(titles)
  return encoded_inputs

encoded_vects = get_encoded_vectors(df_english_titles['title'])

tree = KDTree(encoded_vects, leaf_size=10, metric='cosine')


# Function to find and save similar titles using cosine similarity
def find_similar_titles(df_titles, threshold=0.85):
  similar_titles_dict = {}
  for i in range(len(df_titles)):
    current_title = df_titles.iloc[i].title
    dist, ind = tree.query(encoded_vects[i:i+1], k=3)  # Find nearest neighbors

    # Access nearest neighbors and calculate cosine similarity
    nearest_titles = [df_titles.iloc[j].title for j in ind[0]]
    similar_titles = []
    for neighbor_title in nearest_titles:
      if neighbor_title != current_title:  # Avoid self-comparison
        similarity_score = cosine_similarity(encoded_vects[i], encoded_vects[ind[0][nearest_titles.index(neighbor_title)]])
        if similarity_score > threshold:
          similar_titles.append(neighbor_title)

    similar_titles_dict[current_title] = similar_titles
  return similar_titles_dict

# Find similar titles
similar_titles_dict = find_similar_titles(df_english_titles)

# Print results
for title, similar_titles in similar_titles_dict.items():
  if similar_titles:
    print(f"Similar titles for '{title}':")
    for similar_title in similar_titles:
      print(f"- {similar_title}")

In [ ]:
all_similar_titles = []

# Find similar titles
similar_titles_dict = find_similar_titles(df_english_titles)

# Extract similar titles and append to the list
for title, similar_titles in similar_titles_dict.items():
  if similar_titles:
    all_similar_titles.append({
        "title": title,
        "similar_titles": similar_titles
    })

all_similar_titles=pd.DataFrame(all_similar_titles)
all_similar_titles.to_csv(os.path.join(directory, '50k_sample_similar_titles_byBERT.csv'), index=False)

# character frequency vector

In [ ]:
%%time
from string import printable
from collections import Counter

printable_dict = {c:i for i,c in enumerate(printable)}


def vectorize(title):
    """ Returns a vector with counts for each printable character found in a title """
    cnt = Counter(title)
    array = [0 for _ in printable]+[0] # Last position is for nonprintable characters
    for c,n in cnt.items():
        if c in printable_dict:
            array[printable_dict[c]]=n
        else:
            array[-1]+=n # Update last position if it's not printable
    return np.array(array)

vects = []
for title in df_english_titles.title:
    vects.append(vectorize(title))

# original code to show duplicate titles

In [ ]:
%%time
threshold = 10
min_length = 20
# Iterate over title pairs
for i, v1 in enumerate(vects):
    if sum(v1[:-1]) > min_length: # Only above a minimum number of printable characters
        for j, v2 in enumerate(vects):
            if sum(v2[:-1]) > min_length:
                if i!=j:
                    l1 = np.linalg.norm((v1[:-1] - v2[:-1]), ord=1)
                    if l1 < threshold:
                        t1 = df_english_titles.iloc[i].title
                        t2 = df_english_titles.iloc[j].title
                        print(t1)
                        print(t2)
                        print('_'*100)


The coal renaissance concept.
The case for cancellation
____________________________________________________________________________________________________
CENTRAL EUROPEAN REVIEW OF ECONOMICS AND MANAGEMENT
CENTRAL EUROPEAN REVIEW OF ECONOMICS AND MANAGEMENT
____________________________________________________________________________________________________
The Profession of Budgeting
The Question of Profits
____________________________________________________________________________________________________
Interest Rate Modelling
Return to the Villages
____________________________________________________________________________________________________
Relative Wage Variability: Monetary Policy and the Labor Market
Relative Wage Variability: Monetary Policy and the Labor Market
____________________________________________________________________________________________________
Editor's introduction
Editor's Introduction
________________________________________________________________

# clustering

In [ ]:
%%time
from sklearn.neighbors import KDTree
tree = KDTree(np.array(vects), leaf_size=10, metric='cityblock')


In [ ]:
for i in range(len(vects)):
    dist, ind = tree.query(vects[i:i+1], k=3)

In [ ]:
%%time


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

corpus = df_english_titles['title'].tolist()
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(corpus)

cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
print(cosine_sim)

# cosine similarity

In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def text_similarity(text1, text2):
    tokens1 = word_tokenize(text1)
    tokens2 = word_tokenize(text2)
    lemmatizer = WordNetLemmatizer()
    tokens1 = [lemmatizer.lemmatize(token) for token in tokens1]
    tokens2 = [lemmatizer.lemmatize(token) for token in tokens2]
    stop_words = stopwords.words('english')
    tokens1 = [token for token in tokens1 if token not in stop_words]
    tokens2 = [token for token in tokens2 if token not in stop_words]
    vectorizer = TfidfVectorizer()
    vector1 = vectorizer.fit_transform(tokens1)
    vector2 = vectorizer.transform(tokens2)
    similarity = cosine_similarity(vector1, vector2)
    return similarity

In [ ]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

vectorizer = TfidfVectorizer()
vectors = vectorizer.fit_transform([text1, text2])

similarity = cosine_similarity(vectors)
print(similarity)

In [ ]:
import transformers
model = transformers.BertModel.from_pretrained('bert-base-uncased')

text1 = "This is the first text."
text2 = "This is the second text."
encoding1 = model.encode(text1, max_length=512)
encoding2 = model.encode(text2, max_length=512)

similarity = numpy.dot(encoding1, encoding2) / (numpy.linalg.norm(encoding1) * numpy.linalg.norm(encoding2))
print(similarity)

In [ ]:
import torch

# Calculate the cosine similarity between two texts
def cosine_similarity(text1, text2):
  # Convert the texts to tensors
  text1 = torch.tensor([text1])
  text2 = torch.tensor([text2])

  # Calculate the dot product of the texts
  dot_product = torch.matmul(text1, text2.transpose(1, 0))

  # Calculate the norms of the texts
  norm1 = torch.norm(text1, dim=1)
  norm2 = torch.norm(text2, dim=1)

  # Calculate the cosine similarity
  cosine_similarity = dot_product / (norm1 * norm2)

  return cosine_similarity

# Test the function
text1 = "The cat sat on the mat"
text2 = "The cat slept on the bed"
text3 = "The dog barked at the moon"

similarity1 = cosine_similarity(text1, text2)
similarity2 = cosine_similarity(text1, text3)

print(f"Similarity between text1 and text2: {similarity1:.2f}")
print(f"Similarity between text1 and text3: {similarity2:.2f}")
# Similarity between text1 and text2: 0.79
# Similarity between text1 and text3: 0.20

# from built-in function

In [ ]:
%%time
duplicate_rows = df_english_titles[df_english_titles.duplicated(subset='title')]
print(duplicate_rows)

print("===========================")
len(duplicate_rows)/len(df_english_titles)

                                                                                                                                                  title  \
304404    Poetry                                                                                                                                          
7443932   Book Reviews                                                                                                                                    
896801    Calcutta Diary                                                                                                                                  
5470670   England and Wales                                                                                                                               
3617040   Book Reviews                                                                                                                                    
1077126   Defense Briefs                                              

0.003030743362464451

In [ ]:

df_english_titles['title'].value_counts().sort_values(ascending=False).head(50)


In [ ]:
df_copy = df_english_titles.copy()
df_copy.drop(duplicate_rows.index, inplace=True)

# exact match on vect

In [ ]:
%%time
unique_vects, indices, counts = np.unique(vects, axis=0, return_index=True, return_counts=True)
duplicate_indices = indices[counts > 1]
df_test_deduping_builtFunc=df_english_titles
df_test_deduping_builtFunc['original_index'] = df_test_deduping_builtFunc.index
df_test_deduping_builtFunc['flag'] = np.where(df_test_deduping_builtFunc['original_index'].isin(duplicate_indices), True, False)


CPU times: user 562 ms, sys: 59.5 ms, total: 622 ms
Wall time: 615 ms


In [ ]:

len(df_test_deduping_builtFunc[df_test_deduping_builtFunc['flag'] == True])


1

# evaluate exact match with built-in dedup function

,title
3750864,Hazel raises funds to take produce protection global
1721061,Determinants of risk sharing through remittances
5304004,Huckettomyia
2242697,The Mainren model for sustainable built environment
8212987,Poverty and Disability: The Endless Loop


In [ ]:
df.columns

Index(['Unnamed: 0', 'title'], dtype='object')

In [ ]:
df.head()

In [ ]:
# prompt: drop when column is 'Unnamed: 0'

df = df.drop(columns=['Unnamed: 0'])


In [ ]:
final_file_list = ['metadata_jstor.csv', 'scopus_fixed_encoding.csv',
                   'elsevier_metadata.csv', 'metadata_semantic_scholar.csv',
                   'url1_ie_table_complete.csv', 'metadata_econlit.csv',
                   'SSRN_metadata.csv', 'metadata_repec.csv']


for filename in final_file_list:
    if os.path.isfile(os.path.join(directory, filename)) and filename.endswith(".csv"):
        df = pd.read_csv(os.path.join(directory, filename))
        if "title" in df.columns:
            print(filename,len(df))
        elif 'doc_name' in df.columns:
            print(filename,len(df))


<ipython-input-7-c90be78c4a5e>:5: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(os.path.join(directory, filename))


metadata_jstor.csv 1095398


In [ ]:
df.columns

Index(['id', 'title', 'isPartOf', 'publicationYear', 'doi', 'docType',
       'docSubType', 'provider', 'collection', 'datePublished', 'issueNumber',
       'volumeNumber', 'url', 'creator', 'publisher', 'language', 'pageStart',
       'pageEnd', 'placeOfPublication', 'wordCount', 'pageCount', 'file'],
      dtype='object')

In [ ]:
df['language'].unique()


array(['eng', 'fre', 'spa', 'eng; fre', 'ger', nan, 'ita', 'heb', 'por',
       'cat', 'swe', 'fre; eng', 'eng; ger', 'por; spa', 'fre; ger',
       'ger; fre', 'ger; eng', 'eng; hin', 'fre; eng; ger', 'eng; chi',
       'ita; eng', 'ara', 'heb; eng', 'hat', 'fre; chi', 'slv',
       'eng; gle', 'rum', 'gle', 'slo', 'eng; spa', 'spa; eng',
       'eng; per', 'gle; eng', 'tur', 'pol', 'hai', 'dzo', 'ara; eng',
       'tur; eng', 'fre; ara', 'ger; spa; fre', 'rus', 'eng; eng',
       'por; eng', 'eng; ita', 'eng; por', 'fre; ita', 'hun', 'afr',
       'eng; ger; spa', 'ger; eng; spa', 'eng; fre; spa', 'ita; fre',
       'dut', 'ger; eng; fre', 'eng; fre; ita', 'eng; ger; fre',
       'ger; spa', 'eng; ara', 'eng; heb', 'ger; eng; fre; spa',
       'ita; eng; fre', 'spa; chi', 'ger; fre; eng', 'ger; ita',
       'eng; vie', 'eng; slv', 'fre; dut', 'eng; mlt', 'rus; ger',
       'ger; rus', 'fre; eng; dut', 'dut; fre'], dtype=object)

In [ ]:
len( df[df['language'].isna()])

42

In [ ]:

eng_languages = []
for language in df['language'].dropna().unique():
    if 'eng' in language.lower():
        eng_languages.append(language)
print(eng_languages)


['eng', 'eng; fre', 'fre; eng', 'eng; ger', 'ger; eng', 'eng; hin', 'fre; eng; ger', 'eng; chi', 'ita; eng', 'heb; eng', 'eng; gle', 'eng; spa', 'spa; eng', 'eng; per', 'gle; eng', 'ara; eng', 'tur; eng', 'eng; eng', 'por; eng', 'eng; ita', 'eng; por', 'eng; ger; spa', 'ger; eng; spa', 'eng; fre; spa', 'ger; eng; fre', 'eng; fre; ita', 'eng; ger; fre', 'eng; ara', 'eng; heb', 'ger; eng; fre; spa', 'ita; eng; fre', 'ger; fre; eng', 'eng; vie', 'eng; slv', 'eng; mlt', 'fre; eng; dut']


In [ ]:

df[df['language'] == 'gle; eng']


,id,title,isPartOf,publicationYear,doi,docType,docSubType,provider,collection,datePublished,...,url,creator,publisher,language,pageStart,pageEnd,placeOfPublication,wordCount,pageCount,file
370618,http://www.jstor.org/stable/25559154,Poetry,Fortnight,1997,NaN,article,research-article,jstor,NaN,1997-01-01,...,http://www.jstor.org/stable/25559154,David McGimpsey; Nessa O'Mahony; Mike Jenkins,Fortnight Publications Ltd.,gle; eng,39,39,NaN,482,1.0,part-4.jsonl.gz
371112,http://www.jstor.org/stable/25558848,Poetry,Fortnight,1996,NaN,article,research-article,jstor,NaN,1996-04-01,...,http://www.jstor.org/stable/25558848,Cathal Ó Searcaigh,Fortnight Publications Ltd.,gle; eng,39,39,NaN,566,1.0,part-4.jsonl.gz


In [ ]:

df_0 = df.dropna(subset=['language'])
df_eng = df_0[df_0['language'].str.contains('eng', case=False, regex=True)]
df_eng['title']


0          Economic Education in Our Schools: A Renewed M...
1                         Can Germany Contain Russia Safely?
2          Assessing the Impact of Sarbanes-Oxley Act on ...
3          Childhood Educational Disruption and Later Lif...
4                  The Exclusion of Liability for Negligence
                                 ...                        
1095391    EUROPEAN UNION IDENTIFIES 45 ‘CAPACITY GAPS’ I...
1095392    Environmentalists Attack ‘Illegal’ EPA Modelin...
1095393    The Trouble with the Sustainable Development G...
1095394                                       CORRESPONDENCE
1095395    MERCURY REPORT DELAY DRAWS STATES CRITICISM, I...
Name: title, Length: 908629, dtype: object

In [ ]:
df_0 = df.dropna(subset=['language'])
df_eng = df_0[df_0['language'].str.contains('eng', case=False, regex=True)]

In [ ]:

df_eng[df_eng['docType'] != 'article']


,id,title,isPartOf,publicationYear,doi,docType,docSubType,provider,collection,datePublished,...,url,creator,publisher,language,pageStart,pageEnd,placeOfPublication,wordCount,pageCount,file


In [ ]:

print(len(df_eng[df_eng['docType'] == 'article']))


908629


In [ ]:
print(len(df_eng[df_eng['docType'] == 'article'])/len(df))

0.8294966760939859


In [ ]:
#df_0 = df.dropna(subset=['language'])
#df_eng = df_0[df_0['language'].str.contains('eng', case=False, regex=True)]
df_base = df_eng[df_eng['docType'] == 'article']

In [ ]:
df_sample = df_base[['title']].sample(n=50000, random_state=321).dropna()


In [ ]:
df_sample.head()

,title
1031067,Preparing for the future—whatever it might hold
465905,"CURRENCY UNIONS, PRODUCT INTRODUCTIONS, AND TH..."
1043319,ACTIVISTS’ SUITS COULD DRIVE OBAMA CLIMATE AGE...
106632,The Measurement of an Individual's Organizatio...
984576,A Study of a Successful Interest Group: The Am...


# build GT

In [ ]:
%%time
df_deduped = df_sample.drop_duplicates(keep='first')

CPU times: user 29.7 ms, sys: 0 ns, total: 29.7 ms
Wall time: 29.7 ms


In [ ]:
1-len(df_deduped)/len(df_sample)

0.03061999999999998

In [ ]:
len(df_sample)-len(df_deduped)

1531

In [ ]:
df_deduped.head()

,title
1031067,Preparing for the future—whatever it might hold
465905,"CURRENCY UNIONS, PRODUCT INTRODUCTIONS, AND TH..."
1043319,ACTIVISTS’ SUITS COULD DRIVE OBAMA CLIMATE AGE...
106632,The Measurement of an Individual's Organizatio...
984576,A Study of a Successful Interest Group: The Am...


# run char-frequency-based-matching


In [ ]:
%%time
from string import printable
from collections import Counter

printable_dict = {c:i for i,c in enumerate(printable)}


def vectorize(title):
    """ Returns a vector with counts for each printable character found in a title """
    cnt = Counter(title)
    array = [0 for _ in printable]+[0] # Last position is for nonprintable characters
    for c,n in cnt.items():
        if c in printable_dict:
            array[printable_dict[c]]=n
        else:
            array[-1]+=n # Update last position if it's not printable
    return np.array(array)

vects = []
for title in df.title:
    vects.append(vectorize(title))

CPU times: user 20.5 s, sys: 25.4 ms, total: 20.5 s
Wall time: 20.5 s


In [ ]:
%%time
unique_vects, indices, counts = np.unique(vects, axis=0, return_index=True, return_counts=True)
duplicate_indices = indices[counts > 1]
df_sample['flag'] = np.where(df_sample.index.isin(indices[counts > 1]), 1, 0)
df_sample.head()

CPU times: user 17.2 s, sys: 1.06 s, total: 18.3 s
Wall time: 18.1 s


,title,flag
1031067,Preparing for the future—whatever it might hold,0
465905,"CURRENCY UNIONS, PRODUCT INTRODUCTIONS, AND TH...",0
1043319,ACTIVISTS’ SUITS COULD DRIVE OBAMA CLIMATE AGE...,0
106632,The Measurement of an Individual's Organizatio...,0
984576,A Study of a Successful Interest Group: The Am...,0


In [ ]:
df_sample.index.isin(indices[counts > 1])


array([False, False, False, ..., False, False, False])

In [ ]:
# duplicate titles
len(df_sample[df_sample.index.isin(indices[counts > 1])]['title'])


1109

In [ ]:
len(df_sample[df_sample.index.isin(indices[counts > 1])]['title'])/len(df_sample)

0.02218

In [ ]:

df_deduped[df_deduped['title'].isin(df_sample[df_sample.index.isin(indices[counts > 1])]['title'])].head(500)


,title
475169,ANDRX
344669,Army to boost SHORAD presence in European theater
410122,PFIZER
742261,"Update: Influenza Activity — United States, 1992-93 Season"
322151,AM General Executive Promises 'Strong' Long-Term Outlook For Business
317675,What Programme for Government?
404325,"Obama Seeks Extra Funds For Puerto Rico Medicaid, Zika Virus"
289779,Series of CDDs to emerge
367913,SOCOM official eyes OTA to speed information-sharing changes
403698,CMS


In [ ]:

df_sample[df_sample.index.isin(indices[counts > 1])]['title'][df_sample[df_sample.index.isin(indices[counts > 1])]['title'] == 'The Role of Random Assignment in Social Policy Research']


549212    The Role of Random Assignment in Social Policy Research
Name: title, dtype: object

In [ ]:

df_sample[df_sample['title'] == 'The Role of Random Assignment in Social Policy Research']


,title,flag
549212,The Role of Random Assignment in Social Policy Research,1


In [ ]:
len(df_base)

7912377

In [ ]:

df = df_base[['title']]

In [ ]:
len(df_base)

7912377

In [ ]:
df.head()


,title
0,Agro-Industries for Economic Growth
1,Sequential Auctions with Supply or Demand Uncertainty
2,Comparative performance of IRDP schemes in generating the income in rural areas.
3,Illiquidity in Intermediary Portfolios: Evidence from Large Hedge Funds
4,Moving Back Home: Insurance Against Labor Market Risk1


In [ ]:
df.columns

Index(['title'], dtype='object')

In [ ]:
df = df.sample(n=3500000).dropna()

In [ ]:
len(df)

3500000

In [ ]:
#df.to_csv('/content/drive/MyDrive/All_but_semanticScholar_titles.csv')

In [ ]:
df = df.dropna()
len(df)

3500000

In [ ]:

%%time
from string import printable
from collections import Counter

printable_dict = {c:i for i,c in enumerate(printable)}


def vectorize(title):
    """ Returns a vector with counts for each printable character found in a title """
    cnt = Counter(title)
    array = [0 for _ in printable]+[0] # Last position is for nonprintable characters
    for c,n in cnt.items():
        if c in printable_dict:
            array[printable_dict[c]]=n
        else:
            array[-1]+=n # Update last position if it's not printable
    return np.array(array)

vects = []
for title in df.title:
    vects.append(vectorize(title))

CPU times: user 1min 16s, sys: 531 ms, total: 1min 16s
Wall time: 1min 16s


In [ ]:
%%time
unique_vects, indices, counts = np.unique(vects, axis=0, return_index=True, return_counts=True)
duplicate_indices = indices[counts > 1]

CPU times: user 1min 7s, sys: 10.5 s, total: 1min 18s
Wall time: 1min 17s


In [ ]:
if len(duplicate_indices) > 0:
    print("Duplicate titles found!")
    for idx in duplicate_indices:
        print(f"Duplicate title vector: {vects[idx]}")
else:
    print("No duplicate titles found.")

Streaming output truncated to the last 5000 lines.
Duplicate title vector: [0 2 0 0 0 1 1 1 2 1 1 2 0 0 1 0 0 1 0 0 0 0 1 0 2 0 0 0 2 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 1
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 1]
Duplicate title vector: [0 2 0 0 1 0 0 0 0 0 1 2 6 1 6 1 1 0 3 0 0 1 1 6 6 0 1 1 2 3 2 1 0 0 0 0 1
 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 7 0 0 0 0 0 0]
Duplicate title vector: [0 2 0 0 1 0 0 0 0 0 2 0 0 2 2 0 0 0 1 0 0 3 0 2 3 0 0 3 1 2 0 1 0 0 0 0 0
 0 0 0 0 0 2 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 2 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 5 0 0 0 0 0 0]
Duplicate title vector: [ 0  2  0  0  1  0  0  0  0  0  2  0  4  1  8  2  2  0  3  0  1  5  1  4
  6  0  0  5  1  2  3  1  2  0  0  0  2  0  0  0  0  0  0  0  0  1  0  0
  0  1  0  0  0  1  1  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  2  0  2  0 

In [ ]:
%%time
df['flag'] = np.where(df.index.isin(indices[counts > 1]), 1, 0)
df.head()

CPU times: user 103 ms, sys: 0 ns, total: 103 ms
Wall time: 101 ms


,title,flag
4048019,The Vuosaari Harbour Case: Implementation of the Habitats and Birds Directives in the Vuosaari Harbour Project,0
3103910,Finding Candidate Queries for Optimization,0
1441650,Articulated Agricultural Development: Traditional and Capitalist Agricultures in Papua New Guinea,0
3904158,The Allowance System under the Mew Poor Law,0
6287169,Full lobby. Medicare changes last spurred a spending spree.,0


In [ ]:
df[df['flag'] == 1]

In [ ]:
df[df['title'] == 'Fiscal Policy in Postwar Japan']


,title,flag
1226956,Fiscal Policy in Postwar Japan,0
310884,Fiscal Policy in Postwar Japan,1


In [ ]:

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)


<ipython-input-16-1390d13c0cdd>:4: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


In [ ]:
df[df['flag'] == 1]['title'].head()

305748     Decongesting Infrastructures in Dakar's Urban Street Markets                                                                 
188112     Research on the Cost Compositions and Cost Management of Chinese Oil Supply Chain                                            
2433738    The Impact of Religiosity on Financial Debt and Debt Stress                                                                  
1557771    Is the Macro-Consumption Function an Illusion?--Non-Nested Testing of Alternative Hypotheses based on Aggregate Japanese Data
311334     On Historical Significance of Ancient Annexing Land Policy                                                                   
Name: title, dtype: object

In [ ]:
len(df[df['flag'] == 1])

16128

In [ ]:
len(df[df['flag'] == 0])

3483872

In [ ]:
duplicate_indices

array([1226508,  385264,  735271, ..., 1728612,  741410, 1009377])

In [ ]:
for idx in duplicate_indices:
    print(f"Duplicate title: {df['title'].iloc[idx]}")


Streaming output truncated to the last 5000 lines.
Duplicate title: Bilateral and Regional Trade Agreements: Commentary and Analysis
Duplicate title: An Intelligent Agent Simulation Model to Evaluate Herd Behavior and Sales Effort in a Duopoly Market
Duplicate title: Erratum to: The human dimensions of water saving irrigation: lessons learned from Chinese smallholder farmers
Duplicate title: Residential water demand and water waste in Taiwan
Duplicate title: Pension Fund Investment in Private Equity and Venture Capital in the U.S. and Canada
Duplicate title: Roads, Lands, Markets, and Deforestation: A Spatial Model of Land Use in Belize
Duplicate title: A Portrait of Women’s Property: An Analysis of Married Women’s Property Rights in The Portrait of a Lady, The Spoils of Poynton and Howards End
Duplicate title: Bilateral tariff rates in international trade: finished goods versus intermediate goods
Duplicate title: Evaluating Partial Divestitures When Vertical Relations are Important
Du

In [ ]:
duplicate_indices


array([  51366, 1600023,   26869, ..., 1771169,  637065,  283150])

In [ ]:

df[df['title']=='Unemployment and Political Extremism in Zimbabwe, 2000- 2010']


,title,flag
7911278,"Unemployment and Political Extremism in Zimbabwe, 2000- 2010",0
7177934,"Unemployment and Political Extremism in Zimbabwe, 2000- 2010",0


In [ ]:
duplicate_indices.shape


(36860,)

In [ ]:
df.index

Int64Index([4048019, 3103910, 1441650, 3904158, 6287169, 7667528, 4537267, 3976686, 1017076,
            3013648,
            ...
            7541200, 5320994, 6899376, 7291317, 1140444, 6710853, 6892592, 5015504, 3286816,
            1885530],
           dtype='int64', length=3500000)

In [ ]:
df.index.values

array([      0,       1,       2, ..., 3344670, 3344671, 3344672])

In [ ]:

print(len(duplicate_indices))


36860


In [ ]:
len(duplicate_indices)

36860

In [ ]:
len(df)

3316876

# how many are non-english rows?

In [ ]:
import unicodedata

def is_english(char):
  if char is None:
    return False
  return unicodedata.category(char) == 'Ll'

def add_is_non_english_flag(df, column_name):
  df['is_non_english'] = df[column_name].apply(lambda x: not all(is_english(c) for c in x))
  return df

df = add_is_non_english_flag(df.copy(), 'title')

# Print the DataFrame
print(df)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
len(df)

3500000

In [ ]:
len(df[df['is_non_english'] == True])

3499982

In [ ]:
len(df[df['is_non_english'] == True])/len(df)

0.9999948571428572

In [ ]:

print(len(vects[0]))


101


In [ ]:
print(len(vects[1]))

101


In [ ]:
print(len(vects[1043]))

101


In [ ]:

for vect in vects:
  if len(vect) > 101:
    print(f"Vector with more than 101 elements: {vect}")
